# AWS - RDS MySQL
### Paquetes requeridos
Librería de Python necesaria para establecer conexión a una base de datos MySQL

In [54]:
!pip install pymysql==1.0.2

In [55]:
!pip install sqlalchemy

In [56]:
!pip install mysql-connector-python

### Credenciales
Para acceder a la BD necesitaremos tres datos:
1. Endpoint/host: la dirección donde se encuentra el servidor
2. Usuario
3. Contraseña

In [57]:
host = "database-miercoles.cvyyyacsimzw.eu-north-1.rds.amazonaws.com"
username = "admin"
password = "pruebamiercoles"
port = 3306

### Conexion BD

In [58]:
import pymysql

In [59]:
'''
pymysql.cursors.Dictcursor para que los resultados que devuelva sean diccionarios,
por defecto devuelve tuplas. Asi podemos acceder por clave a las columnas
'''

db = pymysql.connect(host = host,
                     user = username,
                     password = password,
                     cursorclass = pymysql.cursors.DictCursor
                     )

# El objeto cursor es el que ejecutará las queries y devolverá los resultados

cursor = db.cursor()

### Version DB

In [60]:
'''
Este es el engine version de la BD de AWS
fechtone trae la primera linea de la consulta-
El execute() devuelve el numero de filas a las que ha afectado la query,
en este caso, devuelve una unica fila.

Execute se guarda en la conexion pero hasta que no hacemos commit
no se ejecutan las queries que modifiquen la base de datos de alguna manera:
de insert de datos, creación de tablas, etc 
'''

cursor.execute("SELECT VERSION()")
version = cursor.fetchone()
print(f"MysQL version: {version}")

MysQL version: {'VERSION()': '8.0.40'}


### Creación de DB

In [61]:
# Creamos una BD. Tenemos una instancia de MySQL, pero no una BD
# Los comandos de SQL se suelen poner en mayuscula
create_db = '''CREATE DATABASE country_database'''
cursor.execute(create_db)

# Podemos eliminar la BD
# drop_db = '''DROP DATABASE country_database'''
# cursor. execute(drop_db)
# La volvemos a crear
# cursor.execute(create_db)
# el output es el numero de filas afectadas

ProgrammingError: (1007, "Can't create database 'country_database'; database exists")

In [62]:
cursor.execute("SHOW DATABASES")
cursor.fetchall()

[{'Database': 'country_database'},
 {'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'sys'}]

### Creación de tablas

In [63]:
'''
Creamos una tabla 
Esto da error porque no le hemos dicho al servidor que BD
queremos usar
'''

# Crear una tabla que se llame country
create_table = '''
    CREATE TABLE country (
        id INT NOT NULL auto_increment,
        name TEXT,
        continent TEXT,
        population INT,
        gdp DOUBLE,
        primary key (id)
    )
'''
cursor.execute(create_table)

OperationalError: (1046, 'No database selected')

### Seleccionar la BD

In [64]:
# Para usar la BD recien creada
cursor.connection.commit()
use_db = '''USE country_database'''
cursor.execute(use_db)

0

In [65]:
# Chequear todas las tablas que tiene mi db
cursor.execute('SHOW TABLES')
cursor.fetchall()

[{'Tables_in_country_database': 'country'}]

In [66]:
# Si queremos borrar alguna tabla 
# drop_table = '''DROP TABLE country'''
# cursor.execute(drop_table)

### Insertar datos

In [67]:
insert_data = '''
INSERT INTO country(name, continent, population, gdp)
VALUES ('Spain','Europe', 48, 1.62)
'''
cursor.execute(insert_data)

1

In [68]:
name = "Italy"
continent = "Europe"
population = 59
gdp = 2.3

In [69]:
insert_data = f'''
INSERT INTO country(name,continent,population,gdp) VALUES ("{name}","{continent}",{population},{gdp})
'''

In [70]:
cursor.execute(insert_data)

1

### Leer datos

In [71]:
query = '''
SELECT * FROM country WHERE name LIKE "%i%"
'''
cursor.execute(query)

8

In [72]:
print(cursor.fetchall())

[{'id': 1, 'name': 'Spain', 'continent': 'Europe', 'population': 48, 'gdp': 1.62}, {'id': 2, 'name': 'Italy', 'continent': 'Europe', 'population': 59, 'gdp': 2.3}, {'id': 5, 'name': 'Argentina', 'continent': 'America', 'population': 45, 'gdp': 0.7}, {'id': 6, 'name': 'Brazil', 'continent': 'America', 'population': 211, 'gdp': 2.2}, {'id': 9, 'name': 'Argentina', 'continent': 'America', 'population': 45, 'gdp': 0.65}, {'id': 10, 'name': 'Brazil', 'continent': 'America', 'population': 211, 'gdp': 2.17}, {'id': 11, 'name': 'Spain', 'continent': 'Europe', 'population': 48, 'gdp': 1.62}, {'id': 12, 'name': 'Italy', 'continent': 'Europe', 'population': 59, 'gdp': 2.3}]


### Guardar los cambios
Hay que ejecutar el commit antes de cerrar la sesión de la BD para que se guarden todos los cambios

In [ ]:
db.commit()

### Excepciones
Se recomienda rodear este tipo de sentencias con un try/except, ya que suele ser software productivo y tiene que saber manejar fallos

In [ ]:
from pprint import pprint

try:
    sql = '''SELECT * FROM country'''
    cursor.execute(sql)
    pprint(cursor.fetchall(), sort_dicts = False)
except Exception as e:
    print(e)

[{'id': 1,
  'name': 'Spain',
  'continent': 'Europe',
  'population': 48,
  'gdp': 1.62},
 {'id': 2,
  'name': 'Italy',
  'continent': 'Europe',
  'population': 59,
  'gdp': 2.3}]


### Insertar datos de un CSV

In [ ]:
# Creamos el CSV
import pandas as pd

df = pd.read_csv('country_data.csv', sep=';')

In [ ]:
df

,name,continent,population,gdp
0,France,Europe,68,3.05
1,Japan,Asia,124,4.20
2,Argentina,America,45,0.65
3,Brazil,America,211,2.17


In [ ]:
list(zip(*[valor.values() for clave, valor in df.to_dict().items()]))

[('France', 'Europe', 68, 3.05),
 ('Japan', 'Asia', 124, 4.2),
 ('Argentina', 'America', 45, 0.65),
 ('Brazil', 'America', 211, 2.17)]

In [ ]:
for datos in zip(*[valor.values() for clave, valor in df.to_dict().items()]):
    insert = "INSERT INTO country(name,continent,population,gdp) VALUES ('%s', '%s', %d, %.1f)" \
    %(datos[0],datos[1],datos[2],datos[3])
    cursor.execute(insert)

In [ ]:
query = "SELECT * FROM country"
cursor.execute(query)
cursor.fetchall()

InterfaceError: (0, '')

In [ ]:
db.commit()

In [75]:
# Utilizando sqlalchemy
from sqlalchemy import create_engine

# import pymysql
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(user = username, pw = password, host = host, db = "country_database"))

In [76]:
df.to_sql(name = "country", con = engine, if_exists = "append", index = False)

4

In [77]:
db.commit()

InterfaceError: (0, '')

### Cerrar la conexión cuando acabemos

In [78]:
db.close()

Error: Already closed

In [79]:
conn = engine.connect()
conn.close()